In [1]:
## Dependencies
from NanTex_backend.data_postprocessing.oneiros import Oneiros

### Overview
``` Python
# Dreamer
# Dreamer = Oneiros(num_features = 3,               # Number of features in the dataset
#                   DEBUG=True,                     # Debug mode <- leave on for the first run
#                   data_paths_in = None,           # Path to the input data <- in MS Windows, a "None" will trigger a file dialog
#                   data_path_out = None,           # Path to the output data <- in MS Windows, a "None" will trigger a file dialog when you need to save the data
#                   mode = 'has_ground_truth')      # Mode of the data processing <- choose dependent on the data you are about to load in


# Dreamer WINODWS
# -> Dreamer.from_explorer() - this function will call the file explorer dialog to choose the input and output files

## There are two convenience functions that we will use in this notebook:
# - Dreamer.with_ground_truth() - this function will call "from_exploer" with the mode set to "has_ground_truth"
# - Dreamer.without_ground_truth() - this function will call "from_exploer" with the mode set to "no_ground_truth"

# You need to provide the number of present features and if you want to run it in DEBUG morde or not.
# In windows, you can conveniently choose multiple files using the file explorere dialog. 

# Dreamer LINUX
# -> Dreamer.from_glob() - this function will take a number of paths or glob patterns as input and load the data from the files
# Dreamer.from_glob(*['path/to/file1', 'path/to/file2', 'path/to/file3']) - this will load the data from the files
# You sill need to provide the number of present features and if you want to run it in DEBUG morde or not.
```

### Data
Oneiros can read both .npy and image (png, jpeg, jpg) files. If you want to call the Dreamer with ground truth, you need to provide the data as npy files. If you want to call the Dreamer without ground truth, you can provide the data as images. The npy should have the following shape (number_of_features + 1, y_size, x_size). The shape will match if you use the other packages provided within the NanTex project.

### Metaparameters
#### We will use the following metaparameters to run the Dreamer.
``` Json
"feature_static_threshodls" : {     # Static thresholds for each feature
    "feature_0": 0.1,               # Threshold for feature 0
    "feature_1": 0.1,               # Threshold for feature 1
    "feature_2": 0.1                # Threshold for feature 2 <- if you have more features, add them here
    },
"dynamic_thresholds": {             # Dynamic thresholds for each feature <- generates histogram and determins the main population of pixel values. 
                                    # For SMLM this usually is the background and some noise. Be careful using this with images that have a lot of signal.
    "auto_calculate": True,         # Automatically calculate dynamic thresholds using multiples of the standard deviation
    "upper": 3,                     # Upper threshold for dynamic threshold calculation <- hist argmax + 3 bins
    "lower": -2,                    # Lower threshold for dynamic threshold calculation <- hist argmax - 2 bins
    "std_factor": 2                 # Factor to multiply the standard deviation by < hist argmax +- std_factor * std
    },
"patch_size": (256, 256),           # Size of the patches to be extracted from the images
"dream_memory_shape" : None,        # Shape of the memory array for the DREAM algorithm <- will be autmatically populated
"patch_array_shape": None,          # Shape of the patch array <- will be automatically populated
"standardize": True,                # Standardize the patches
"normalize": False,                 # Normalize the patches <- not recommended
"tensortype": torch.float32,        # Type of tensor to be used
"out_type": np.uint8,               # Determine the type of the output tensor <- usually np.uint8
"weights_only": True,               # DEBUG parameter when loading a state_dict (torch) <- leave as it
"cast_all_to_img": True,            # Cast all tensors to images will normalize the output and cast it to the maximum range of the output type
"append_original_features": True,   # Append the original features to the output tensor <- for your convenience when exporting
"append_original_overlays": True,   # Append the original overlays to the output tensor
"append_dream_overlays": True,      # Append the dream overlays to the output tensor
"apply_static_thresholds": True,    # Apply the static thresholds to dream features 
"apply_dynamic_thresholds": True    # Apply the dynamic thresholds to dream features
```

#### The metaparameters are stored in a dictionary and can be accessed and changed using the following code:
``` Python
# Access the metaparameters
metaparameters = Dreamer.metaparameters

# Change the metaparameters
metaparameters['patch_size'] = (256, 256)

# Update the metaparameters
new_metaparameters = {
    'patch_size': (256, 256),
    'some_other_metaparameter': 'some_value'                    
                     }
Dreamer.metaparameters.update(new_metaparameters)
```


# LINUX

In [7]:
# Grab filepaths
from glob import glob

root:str = 'path/to/your/data'
paths:list = glob(root + '/*.npy')
paths = paths [:10] # <- for testing purposes

In [ ]:
# set up the dreamer for LINUX
Dreamer = Oneiros.from_glob(*paths,
                            num_features = 2,
                            num_channels_out = 3,
                            DEBUG=True,
                            mode = 'has_ground_truth')

In [ ]:
# load the model
Dreamer.jumpstart_model(state_dict_path = "path/to/model.pt") 

In [ ]:
# if you want to load in more data without reinitializing the model
Dreamer.load_new_data_linux("path/to/new/data_1.npy", "path/to/new/data_2.npy", ...)

In [ ]:
Dreamer.dream() # <- this will start the data processing

In [ ]:
# Visualize the first dream

# stylesheet
from NanTex_backend.Util.pltStyler import pltStyler
pltStyler().enforece_stylesheet()

Dreamer.visualize(dream_no = 5,             # <- dream number to visualize
                  cmap = 'inferno',         # <- colormap to use, defaults to 'gray
                  ticks = False,            # <- on/off ticks
                  return_fig_axs = False)   # <- return fig and axs objects

In [ ]:
# Export your results
# npy will export each dream to a 4D tensor
# png will export each dream to a folder of pngs

Dreamer.export(out_type='png', # <- output type, choose from 'png' or 'npy' or implement your own
               outpath='.')    # <- output path

# Windows

## With Ground Truth

In [ ]:
# set up the dreamer
Dreamer = Oneiros.with_ground_truth(num_features = 2,
                                    num_channels_out = 2,
                                    DEBUG=True)

In [ ]:
# Quickstart model
# Just choose the torch checkpoint file and the model will be loaded
Dreamer.jumpstart_model()

In [ ]:
## load new data without reloading the model
Dreamer.load_new_data_windows()

In [4]:
# change patchsize
Dreamer.metadata['patch_size'] = (256, 256)

In [ ]:
Dreamer.dream()

In [ ]:
## Get an overview of the metaparameters
from pprint import pprint
pprint(Dreamer.metadata)

In [ ]:
## Play with the static thresholds to delete background noise
Dreamer.metadata['feature_static_threshodls'].update({
    'feature_1': 0.1,
    'feature_2': 0.1,
    'feature_3': 0.1,
})
# Play with the dynamic thresholds to delete structural noise
Dreamer.metadata['dynamic_thresholds'].update({
    'auto_calculate': False,
    'upper': 3,
    'lower': -2,
    'std_factor':2
})


# Play with the dreamer parameters
Dreamer.metadata['apply_dynamic_thresholds'] = True
Dreamer.metadata['apply_static_thresholds'] = True

# choose what extras to export
Dreamer.metadata['append_dream_overlays'] = True
Dreamer.metadata['append_original_overlays'] = True
Dreamer.metadata['append_original_features'] = True

Dreamer.metadata['cast_all_to_img'] = True # if you dont want to export straight to image

# however, you have to re-run the post-processing
Dreamer.__post_process_data__()

In [ ]:
Dreamer.data_out['dream_0'].keys()

In [ ]:
Dreamer.data_out['dream_0']

In [ ]:
## Plot and vsualize the results
from NanTex_backend.Util.pltStyler import pltStyler
pltStyler().enforece_stylesheet()
Dreamer.visualize(dream_no = 0,
                  return_fig_axs=False)

In [ ]:
## Export the results
Dreamer.export(out_type='npy') # png <- image, npy <- numpy array, if you want mire, you need to implement it

## Without Ground Truth

In [ ]:
# set up the dreamer
Dreamer = Oneiros.without_ground_truth(num_features = 2,
                                       num_channels_out = 2,
                                       DEBUG=True)

In [ ]:
Dreamer.jumpstart_model()

In [ ]:
Dreamer.load_new_data_windows()

In [ ]:
Dreamer.dream()

In [ ]:
from NanTex_backend.Util.pltStyler import pltStyler
pltStyler().enforece_stylesheet()
Dreamer.visualize(0, return_fig_axs=False, cmap = 'inferno')

In [ ]:
## Export the results
Dreamer.export(out_type='png') # png <- image, npy <- numpy array, if you want mire, you need to implement it

## Deeper insights and step by step execution

In [ ]:
## Test Pre-Processing
Dreamer.__pre_process_data__()

In [ ]:
## Dependencies
from NanTex_backend.deep_learning.dl_model_assembly import assembled_model, final_layer_config

## Load model
Dreamer.setup_model(model=assembled_model['model'],
                    activation=assembled_model['activation'],
                    device=assembled_model['device'],
                    state_dict_path=None)
Dreamer.load_weights(None)

In [ ]:
Dreamer.__offload_data_to_device__()

In [ ]:
Dreamer.metadata

In [ ]:
Dreamer.__go_to_sleep__()

In [ ]:
Dreamer.__post_process_data__()

In [ ]:
## Visualization
import numpy as np
def normalize(x):
    return (x - np.min(x)) / (np.max(x) - np.min(x))

# Dependencies
from NanTex_backend.Util.pltStyler import pltStyler
import matplotlib.pyplot as plt

# control
Select_Dream:int = 0

# layout
pltStyler().enforece_stylesheet()

# figure
fig, axs = plt.subplots(1, final_layer_config['out_channels'] + 1, figsize=(5*(final_layer_config['out_channels'] + 1), 5))

# plot features
for i, key in enumerate(Dreamer.data_out[f"dream_{Select_Dream}"].keys()):
    if key in ['original_overlay', 'dream_overlay'] + [f'original_feature_{i}' for i in range(1, 4)]:
        continue
    
    axs[i].imshow(normalize(Dreamer.data_out[f"dream_{Select_Dream}"][key]) * 255, cmap='gray')
    axs[i].set_title(f"{key}")
    axs[i].tick_params(axis='both', 
                       which='both', 
                       bottom=False, 
                       top=False, 
                       left=False, 
                       right=False, 
                       labelbottom=False, 
                       labelleft=False)
    
# add original
axs[-1].imshow(normalize(Dreamer.data_out[f"dream_{Select_Dream}"]['original_overlay']) * 255, cmap='gray')
axs[-1].set_title('Original')